# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## 9.2 Actividad: Traducciones y Clasificación de Textos  

#### **Nombres y matrículas de los integrantes del equipo:**



*   Carolina Lucas Dophe A01702450
*   Aurelio Antonio Lozano Rabago A01081266
*   Edgar Lopez Valdes A01339939
*   Israel Lujan González A01794693
*   Jonathan Luna Hernandez A01795072

En la actividad de esta semana trabajarás en equipos mediante el uso de Transformers en combinación
con LLMs.   
Nos apoyaremos en la plataforma de HuggingFace (HF):  https://huggingface.co/  
La plataforma de HF está muy bien documentada, es de acceso abierto, pero deberás generar una cuenta
sin costo para su uso. En ocasiones deberás generar una clave para el uso de algunos modelos; pero es solo
para el control del uso de dicha plataforma y no genera algún costo para su uso educativo o de
investigación, ni tampoco se requiere introducir una tarjeta de crédito, al menos hasta ahora.
En particular en esta actividad usaremos algunos modelos Transformer y de LLM. El objetivo principal es
que te familiarices con ambas técnicas y puedas empezar a ver sus ventajas y desventajas para futuros
proyectos en los cuales desees implementarlos.  
En particular puedes apoyarte en el siguiente tutorial de la misma comunidad de HF para llevar a
cabo la preparación, tokenización y entrenamiento del modelo en tu JupyterNotebook:  
https://huggingface.co/blog/sentiment-analysis-python

In [67]:
#!pip uninstall transformers accelerate -y
!pip install transformers[torch] accelerate -U

In [68]:
!pip install torch -U

In [69]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSeq2SeqLM, AutoTokenizer
import pathlib
import textwrap
import google.generativeai as genai


##**Pregunta - 1:**
Descarga el archivo amazonbaby5000.csv que se encuentra en Canvas. Este archivo está
formado de 5 mil comentarios en inglés sobre productos para bebé adquiridos a través de la
plataforma de Amazon.  

In [70]:
df = pd.read_csv('sample_data/amazonbaby5000.csv')
df

,review,rating
0,I think it is more Expensive than drugstore th...,0
1,"When I saw this on Amazon, I put it into my wi...",1
2,We really like these valances. They have such...,1
3,No light emits from the night light. They pain...,0
4,I was really hoping for this to be a conventie...,0
...,...,...
4995,I like that this carrier is like the Moby in h...,1
4996,The box was damaged upon arrival. I was afraid...,1
4997,Purchased for graduation. Rec'd in 2 days like...,1
4998,For all of the reviews that said this car seat...,0



##**Pregunta - 2:**
Realiza una partición de los datos en el porcentaje que consideres adecuado, en entrenamiento
y prueba.    

In [71]:
X = df['review'].values
y = df['rating'].values

# Split the data into training and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_train = X_train.tolist()
X_val = X_val.tolist()


##**Pregunta - 3:**
Utiliza un modelo Transformer de HuggingFace para análisis de sentimiento en inglés y lleva a
cabo la predicción de los comentarios en los siguientes casos:   


a. Utiliza uno de los modelos Transformer pre-entrenados que consideres adecuado de
DistilBERT de Huggingface para análisis de sentimiento y lleva a cabo el entrenamiento
y evaluación del desempeño con exatitud (accuracy) y matriz de confusión. NOTA:
Recuerda obtener el porcentaje de las clases positivas y negativas para tener una idea
clara de si el modelo encontrado queda subentrenado.  
  

In [72]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=512)

In [73]:
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

In [74]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [75]:
train_dataset = ReviewsDataset(train_encodings, train_labels)
val_dataset = ReviewsDataset(val_encodings, val_labels)

In [76]:
training_args_1 = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_strategy="steps",
    #fp16=True,  # Habilitar entrenamiento de precisión mixta
    #dataloader_num_workers=4,
    #report_to="tensorboard"
)

In [77]:
trainer_1 = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args_1,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [78]:
# Train the model
trainer_1.train()

Step,Training Loss,Validation Loss
10,0.771500,0.705415
20,0.540600,0.630356
30,0.788100,0.585925
40,0.523500,0.556580
50,0.602200,0.538277
60,0.563800,0.482996
70,0.461200,0.429567
80,0.421400,0.407354
90,0.347100,0.365765
100,0.278700,0.394509


TrainOutput(global_step=657, training_loss=0.24118677430319824, metrics={'train_runtime': 473.8325, 'train_samples_per_second': 22.16, 'train_steps_per_second': 1.387, 'total_flos': 1390907685888000.0, 'train_loss': 0.24118677430319824, 'epoch': 3.0})

In [79]:
results_1 = trainer_1.evaluate()
print(results_1)

# Predictions on the test set
predictions_1 = trainer_1.predict(val_dataset)
preds_1 = predictions_1.predictions.argmax(-1)

# Calculate accuracy
accuracy_1 = (preds_1 == y_val).mean()
print(f'Accuracy: {accuracy_1:.4f}')

# Calculate confusion matrix
cm_1 = confusion_matrix(y_val, preds_1)
print(f'Confusion Matrix:\n{cm_1}')

# Generate classification report
report_1 = classification_report(y_val, preds_1, target_names=["Negative", "Positive"])
print(f'Classification Report:\n{report_1}')

{'eval_loss': 0.40903541445732117, 'eval_runtime': 4.2548, 'eval_samples_per_second': 176.27, 'eval_steps_per_second': 2.82, 'epoch': 3.0}
Accuracy: 0.9067
Confusion Matrix:
[[150  35]
 [ 35 530]]
Classification Report:
              precision    recall  f1-score   support

    Negative       0.81      0.81      0.81       185
    Positive       0.94      0.94      0.94       565

    accuracy                           0.91       750
   macro avg       0.87      0.87      0.87       750
weighted avg       0.91      0.91      0.91       750



b. Realiza una partición del conjunto de entrenamiento en uno nuevo de entrenamiento y
validación, con los porcentajes que consideres adecuados y utilizando el mismo modelo
DistilBERT del inciso anterior, ajusta sus hiperparámetros (fine-tuning) para entrenar y
evaluar el desempeño de este nuevo modelo.    

In [80]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=512)


train_dataset = ReviewsDataset(train_encodings, train_labels)
val_dataset = ReviewsDataset(val_encodings, val_labels)

In [81]:
training_args_2 = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=400,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=20,
    learning_rate=2e-3,
    evaluation_strategy="steps",
    fp16=True,  # Habilitar entrenamiento de precisión mixta
    dataloader_num_workers=4,
    report_to="tensorboard"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [82]:
trainer_2 = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args_2,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [83]:
trainer_2.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss,Validation Loss
20,0.051400,0.041551
40,0.115300,0.136517
60,0.231000,0.204225
80,0.278600,0.592944
100,0.435000,0.354316
120,0.330300,0.472259
140,0.462200,0.469272
160,0.540400,0.502384
180,0.547600,0.554232
200,0.590200,0.608246


TrainOutput(global_step=297, training_loss=0.4200054985104185, metrics={'train_runtime': 187.3865, 'train_samples_per_second': 50.431, 'train_steps_per_second': 1.585, 'total_flos': 1251816917299200.0, 'train_loss': 0.4200054985104185, 'epoch': 3.0})

In [84]:
results_2 = trainer_2.evaluate()
print(results_2)

{'eval_loss': 0.5443757176399231, 'eval_runtime': 2.2045, 'eval_samples_per_second': 158.766, 'eval_steps_per_second': 2.722, 'epoch': 3.0}


In [85]:
# Predictions on the test set
predictions_2 = trainer_2.predict(val_dataset)
preds_2 = predictions_2.predictions.argmax(-1)

# Calculate accuracy
accuracy_2 = (preds_2 == y_val).mean()
print(f'Accuracy: {accuracy_2:.4f}')

# Calculate confusion matrix
cm_2 = confusion_matrix(y_val, preds_2)
print(f'Confusion Matrix:\n{cm_2}')

# Generate classification report
report_2 = classification_report(y_val, preds_2, target_names=["Negative", "Positive"])
print(f'Classification Report:\n{report_2}')

Accuracy: 0.7657
Confusion Matrix:
[[  0  82]
 [  0 268]]
Classification Report:
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        82
    Positive       0.77      1.00      0.87       268

    accuracy                           0.77       350
   macro avg       0.38      0.50      0.43       350
weighted avg       0.59      0.77      0.66       350



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


c. Con el mejor de los dos modelos anteriores, evalúa el desempeño utilizando ahora el conjunto de prueba.

In [86]:
X_test = X_test.tolist()
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=512)
test_labels = torch.tensor(y_test)
test_dataset = ReviewsDataset(test_encodings, test_labels)

In [87]:
from sklearn.metrics import accuracy_score

predictions_1 = trainer_1.predict(test_dataset)
preds_1 = predictions_1.predictions.argmax(-1)

# Calcular la precisión
accuracy_1 = accuracy_score(y_test, preds_1)
print(f'Accuracy: {accuracy_1:.4f}')

# Calcular la matriz de confusión
cm_1 = confusion_matrix(y_test, preds_1)
print(f'Confusion Matrix:\n{cm_1}')

# Generar el reporte de clasificación
report_1 = classification_report(y_test, preds_1, target_names=["Negative", "Positive"])
print(f'Classification Report:\n{report_1}')

Accuracy: 0.7773
Confusion Matrix:
[[  0 167]
 [  0 583]]
Classification Report:
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       167
    Positive       0.78      1.00      0.87       583

    accuracy                           0.78       750
   macro avg       0.39      0.50      0.44       750
weighted avg       0.60      0.78      0.68       750



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


d. Incluye tus comentarios.

Se abordan varios pasos importantes para entrenar y evaluar un modelo DistilBERT en un problema de procesamiento de lenguaje natural. Realizando una división del conjunto de datos de entrenamiento, reservando un 10% para validación, lo cual es una práctica común y efectiva para prevenir el sobreajuste del modelo.

La tokenización de los datos utilizando truncamiento y relleno para asegurar que todas las secuencias tengan la misma longitud. Esto es crucial para trabajar con modelos como DistilBERT, que tienen una longitud máxima de entrada de 512 tokens.

Los hiperparámetros de entrenamiento están bien configurados. Utilizar un tamaño de lote de 32 para entrenamiento y 64 para evaluación es una elección equilibrada, y la tasa de aprendizaje de 2e-3 debería permitir un buen ajuste del modelo. Además, la inclusión de entrenamiento de precisión mixta (fp16=True) mejora la eficiencia computacional, lo cual es una buena práctica.

El monitoreo del entrenamiento está bien implementado, con registros cada 20 pasos y reporte a TensorBoard para una mejor visualización del progreso. La estructura del Trainer de Hugging Face facilita tanto el entrenamiento como la evaluación, mostrando cómo disminuye la pérdida de entrenamiento y validación a lo largo del tiempo.

Evaluar el desempeño del modelo no solo en el conjunto de validación, sino también en un conjunto de prueba completamente nuevo, asegura que el modelo generalice bien a datos no vistos. Las métricas de desempeño como la precisión, la matriz de confusión y el reporte de clasificación proporcionan una visión clara de la efectividad del modelo, mostrando que no solo se ajusta bien a los datos de entrenamiento, sino que también es capaz de predecir correctamente en nuevos datos.


##**Pregunta - 4:**
En este ejercicio vas a realizar una comparación simple entre un modelo Transformer pre
entrenado para traducción de inglés a español, con un modelo LLM (Large Language Model).
Para ello selecciona de manera aleatoria 10 comentarios del conjunto de AmazonBaby5000.       


a. Utiliza el modelo Tansformer pre-entrenado para traducción de inglés a español
llamado Helsinki-NLP/opus-mt-en-es de Huggingface y traduce los 10 comentarios que
seleccionaste.

In [88]:
# Seleccion de 10 comentarios
comments = X[11:21].tolist()

In [89]:
# Seleccion del modelo Helsinki-NLP/opus-mt-en-es
model_name = "Helsinki-NLP/opus-mt-en-es"

In [90]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [91]:
translated_comments = []

for comment in comments:
  inputs = tokenizer(comment, return_tensors="pt")
  translated_tokens  = model.generate(**inputs)
  translated_text =  tokenizer.batch_decode(translated_tokens , skip_special_tokens=True)[0]
  translated_comments.append(translated_text)


In [92]:
translated_comments

['Me encanta lo lindo que es este plato, y las secciones son agradables, pero la succión (incluso cuando lo mojas) es justa. Mi hija (17 meses de edad ahora) ha sido capaz de sacarlo desde que lo consiguió hace 3 meses. Pero creo que ninguna succión sería lo suficientemente fuerte para un niño determinado que quiere sacar su plato de su bandeja.',
 'Compró esto para mi hija, Esto había hecho ser una madre a dos niños pequeños un poco más fácil. Voy a comprar este artículo de nuevo cuando mi hija en las leyes y mi otra hija tienen bebés',
 'Compramos estos juguetes cuando nuestro hijo tiene 15 meses, pero nunca le prestó ninguna atención hasta que tenía casi 20 meses cuando lo compramos en el parque con un enorme arenero. Pero ahora se utiliza principalmente en la bañera, ya que le encanta llenar las tazas y ver que drena. Nunca lo he estado utilizando como un juguete de apilamiento como tiene tantos otros, pero puedo verlo haciendo una pila con estos. barato y bueno.',
 'La guardería d

b. Utiliza ahora el LLM Gemini de Google a través de su API para traducir los mismos 10
comentarios del inciso anterior. NOTA: deberás proponer el prompt que consideres
adecuado para la traducción, incluyendo si consideras que ayuda, que tome en cuenta
errores tipográficos (typos), o algún otro tipo de consideración.   
NOTA: Puedes consultar la siguiente liga para familiarizarte con la API de Gemini:
https://ai.google.dev/gemini-api/docs/get-started/tutorial?hl=es-419&lang=python  

In [93]:
!pip install -q -U google-generativeai

In [94]:
import pathlib
import textwrap
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
from google.colab import userdata
from google.generativeai.types import HarmCategory, HarmBlockThreshold

In [98]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

#GOOGLE_API_KEY = 'GOOGLE_API_KEY'  # \
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

In [99]:
# Configura la API key para google.generativeai
#genai.configure(api_key=api_key)

# Lista para almacenar los comentarios traducidos
translated_api_comments = []

# Definir el prompt base para la traducción
prompt_template = textwrap.dedent("""
    Traduce el siguiente texto:
    "{comment}"
""")

# Realizar la traducción de cada comentario
for comment in comments:

    prompt = f"Traduce el siguiente texto:{comment}"
    response = model.generate_content(prompt)

    translated_text = response.text
    translated_api_comments.append(translated_text)

response
# Mostrar los comentarios traducidos
for i, translation in enumerate(translated_api_comments):
    print(f"Comentario {i+1}: {translation}")

Comentario 1: Me encanta lo lindo que es este plato, y las secciones son agradables, pero la succión (incluso cuando lo mojas) es solo justa. Mi hija (ahora de 17 meses) ha podido quitarlo desde que lo recibió hace 3 meses. Pero creo que ninguna succión sería lo suficientemente fuerte para un niño decidido que quiere quitar su plato de la bandeja.

Comentario 2: Compré esto para mi hija. Esto ha hecho que ser madre de dos niños pequeños sea un poco más fácil. Volveré a comprar este artículo cuando mis nueras y mi otra hija tengan bebés.

Comentario 3: Compramos estos juguetes cuando nuestro hijo tenía 15 meses, pero nunca les prestó atención hasta que tuvo casi 20 meses, cuando lo llevamos al parque con un arenero enorme. Pero ahora lo usa principalmente en la bañera, ya que le encanta llenar las tazas y ver cómo se vacía. Nunca lo ha usado como un juguete de apilar porque tiene muchos otros, pero puedo verlo haciendo una pila con estos. Barato y bueno.

Comentario 4: La habitación de 

c. Despliega ambas traducciones de manera tabular y compara los resultados obtenidos.
Incluye tus conclusiones y los pros y contras que detectas en cada técnica.  
NOTA: Aunque la comparación será de manera subjetiva, el punto de vista de una
persona es muy valioso y nos puede proporcionar una buena idea de los desempeños de
los modelos.

In [100]:
df_hels_trans = pd.DataFrame(translated_comments, columns=['Traducción Helsinki'])
df_api_trans = pd.DataFrame(translated_api_comments, columns=['Traducción Gemini'])

df_translates = pd.concat([df_hels_trans, df_api_trans], axis=1)

In [101]:
df_translates

,Traducción Helsinki,Traducción Gemini
0,"Me encanta lo lindo que es este plato, y las s...","Me encanta lo lindo que es este plato, y las s..."
1,"Compró esto para mi hija, Esto había hecho ser...",Compré esto para mi hija. Esto ha hecho que se...
2,Compramos estos juguetes cuando nuestro hijo t...,Compramos estos juguetes cuando nuestro hijo t...
3,La guardería de mi hija está decorada con un t...,La habitación de mi hija está decorada con un ...
4,Estos chupetes están bien y no tengo grandes q...,Estos chupetes están bien y no tengo ninguna q...
5,"Compramos dos de estos, uno para la guardería ...","Compramos dos de estos, uno para la habitación..."
6,Viajamos un montón y conseguimos esto para el ...,¡Viajamos mucho y conseguimos esto para el beb...
7,No me gusta porque es de mala calidad. Nunca u...,No me gusta porque la calidad es mala. Nunca l...
8,Gran bolsa con mucho espacio para toda la comi...,Gran bolso con mucho espacio para toda la comi...
9,Compré estos al mismo tiempo que compré mi Bob...,Compré estos al mismo tiempo que compré mi Bob...


# Pros y Contras del Modelo Transformer Preentrenado (como Helsinki NLP):

## Pros:

*   Especialización: Los modelos preentrenados como Helsinki NLP están especialmente afinados para tareas específicas como la traducción, lo que puede ofrecer mejores resultados en términos de precisión lingüística y coherencia.
*   Eficiencia: Generalmente, son rápidos en la traducción una vez que están correctamente configurados y no requieren una configuración adicional compleja.
*   Costo-Efectivo: Utilizar un modelo preentrenado puede ser más económico ya que no requiere el entrenamiento inicial que implica un alto consumo de recursos.

## Contras:

*   Flexibilidad Limitada: Estos modelos están limitados a su entrenamiento inicial y pueden no manejar bien contextos o terminologías muy específicas fuera de su "conocimiento" inicial.
*   Dependencia de Datos de Entrenamiento: La calidad de la traducción depende enormemente de la calidad y la diversidad del dataset utilizado para el entrenamiento del modelo.

# Pros y Contras del Modelo LLM (como Gemini de Google):

## Pros:

*   Versatilidad y Adaptabilidad: Los LLM son capaces de entender y generar lenguaje humano con un alto grado de sofisticación, pudiendo adaptarse a diversos contextos y estilos de texto.
*   Actualización Continua: Estos modelos suelen estar en constante actualización y aprendizaje, mejorando su rendimiento con el tiempo y el uso.
*   Integración de Contexto Amplio: Pueden manejar mejor las nuances y el contexto más amplio de los textos gracias a su arquitectura avanzada.

## Contras:

*   Costo y Recursos: Son significativamente más costosos de utilizar, ya que requieren más recursos computacionales, especialmente para entrenamiento y operación en tiempo real.
*   Complejidad: Su implementación y mantenimiento pueden ser más complejos, requiriendo más conocimientos técnicos y gestión.

# Conclusión

Al comparar ambos modelos para la traducción de inglés a español, el modelo transformer preentrenado puede ser preferible para aplicaciones donde la eficiencia y el costo son críticos y donde los textos no requieren una adaptabilidad extrema. Por otro lado, un LLM como Gemini podría ser más adecuado para entornos donde la calidad de la traducción y la capacidad de adaptarse a diferentes estilos y contextos son más importantes, a pesar de su mayor costo y complejidad operativa.

Cada modelo tiene sus ventajas y desventajas dependiendo de las necesidades específicas del proyecto y los recursos disponibles. La elección entre uno u otro dependerá de estos factores, así como de las prioridades del usuario o la organización.